# Linear Programming Tube Cutting
Using PuLP for linear programming in python. Solving a basic material optimization problem. Determine which parts should be cut on which piece of stock such that scrap material is minimized. All parts and stock are tubes which only have a length. This problem is similar to a 1d bin packing problem.

Variables:
- $ l_i $ = Length of part i. i = 1, 2, 3, ... n
- $ M_j $ = Length of stock j. j = 1, 2, 3, ... m
- $ x_{i,j} $ = {1 if part i is cut on stock j, 0 otherwise}
- $ y_j $ = {1 if stock j is used, 0 otherwise}

Objective:
- Minimize the excess stock material after all parts are cut.

$$ min \sum_{j=1}^{m} (M_j * y_j - \sum_{i=1}^{n} l_i * x_{i,j}) $$

Subject to:
- Total length of parts cut can not exceed stock length used.
- All parts must be cut once

$$ M_j * y_j - \sum_{i=1}^{n} l_i * x_{i,j} \geq 0 \quad \forall j$$
$$ \sum_{j=1}^{m} x_{i,j} = 1 \quad \forall i$$

Note:
- There is an implied condition that $ x_{i,j} $ can only be 1 if $ y_j $ is also 1 for the given j.
- There will be an error if tubes can't be cut

In [196]:
import pulp
import pandas as pd

In [197]:
# .049 data
# part_df = pd.read_csv('data/0.049_part_data.csv')
# stock_df = pd.read_csv('data/0.049_stock_data.csv')

# # .065 data
part_df = pd.read_csv('data/0.065_part_data.csv')
stock_df = pd.read_csv('data/0.065_stock_data.csv')

part_list = part_df["Length"].to_list()
stock_list = stock_df["Length"].to_list()

# Debug part and stock list
# part_list = [1, 2, 3, 4, 5, 10, 15]
# stock_list = [10, 30, 35]

In [198]:
# Could also use unique part/stock number as index, dict contains index and length
part_dict = {i:length for i, length in enumerate(part_list)}
stock_dict = {i:length for i, length in enumerate(stock_list)}

In [199]:
part_dict

{0: 13.27,
 1: 13.27,
 2: 30.5,
 3: 30.5,
 4: 7.41,
 5: 7.41,
 6: 8.25,
 7: 8.25,
 8: 13.92,
 9: 13.92,
 10: 30.25,
 11: 30.25,
 12: 18.15,
 13: 18.15,
 14: 15.3,
 15: 15.3,
 16: 29.77,
 17: 29.77,
 18: 13.5}

In [200]:
stock_dict

{0: 66, 1: 66, 2: 66, 3: 66, 4: 66, 5: 66, 6: 66, 7: 54}

In [201]:
def optimize_tube_cut(part_dict:dict, stock_dict:dict) -> tuple[pulp.LpProblem, dict, dict]:
    '''
    Optimizes part and stock allocation for tube cutting. 
    '''
    prob = pulp.LpProblem("Tube_Cut", pulp.LpMinimize)
    
    # Variables
    part_vars = pulp.LpVariable.dicts("Part_Stock", (part_dict.keys(), stock_dict.keys()), 0, 1, cat= pulp.LpInteger)
    stock_vars = pulp.LpVariable.dicts("Stock", stock_dict.keys(), 0, 1, cat= pulp.LpInteger)
    
    # Objective
    prob += (pulp.lpSum([stock_dict[j] * stock_vars[j] - 
                        pulp.lpSum([part_dict[i] * part_vars[i][j] for i in part_dict.keys()])
                        for j in stock_dict.keys()]), 
                        "Excess Stock")
    # Subject to
    for j in stock_dict.keys():
        prob += (pulp.lpSum([part_dict[i] * part_vars[i][j] for i in part_dict.keys()]) - stock_dict[j] * stock_vars[j] <= 0, 
                ("Stock Capacity " + str(j)))
    for i in part_dict.keys():
        prob += (pulp.lpSum([part_vars[i][j] for j in stock_dict.keys()]) - 1 == 0, 
                ("Part Cut " + str(i)))
    prob.solve()
    
    return prob, part_vars, stock_vars

In [202]:
prob_result, part_vars_results, stock_vars_results = optimize_tube_cut(part_dict, stock_dict)
print("Status:", pulp.LpStatus[prob_result.status])

Status: Optimal


In [203]:
for j in stock_dict.keys():
    if stock_vars_results[j].varValue != 0:
        print(str(stock_vars_results[j]) + ": " + str(stock_vars_results[j].varValue))
    for i in part_dict.keys():
        if part_vars_results[i][j].varValue != 0:
            print(str(part_vars_results[i][j]) + ": " + str(part_vars_results[i][j].varValue))

print("Excess Material = ", pulp.value(prob_result.objective))

Stock_0: 1.0
Part_Stock_4_0: 1.0
Part_Stock_5_0: 1.0
Part_Stock_11_0: 1.0
Part_Stock_18_0: 1.0
Stock_1: 1.0
Part_Stock_7_1: 1.0
Part_Stock_15_1: 1.0
Part_Stock_17_1: 1.0
Stock_2: 1.0
Part_Stock_1_2: 1.0
Part_Stock_3_2: 1.0
Part_Stock_13_2: 1.0
Stock_3: 1.0
Part_Stock_2_3: 1.0
Part_Stock_10_3: 1.0
Stock_6: 1.0
Part_Stock_8_6: 1.0
Part_Stock_14_6: 1.0
Part_Stock_16_6: 1.0
Stock_7: 1.0
Part_Stock_0_7: 1.0
Part_Stock_6_7: 1.0
Part_Stock_9_7: 1.0
Part_Stock_12_7: 1.0
Excess Material =  36.860000000000035


In [204]:
for j in stock_dict.keys():
    if stock_vars_results[j].varValue != 0:
        print("Stock " + str(j) + ": " + str(stock_dict[j]))
    for i in part_dict.keys():
        if part_vars_results[i][j].varValue != 0:
            print("Part " + str(i) + ": " + str(part_dict[i]))

Stock 0: 66
Part 4: 7.41
Part 5: 7.41
Part 11: 30.25
Part 18: 13.5
Stock 1: 66
Part 7: 8.25
Part 15: 15.3
Part 17: 29.77
Stock 2: 66
Part 1: 13.27
Part 3: 30.5
Part 13: 18.15
Stock 3: 66
Part 2: 30.5
Part 10: 30.25
Stock 6: 66
Part 8: 13.92
Part 14: 15.3
Part 16: 29.77
Stock 7: 54
Part 0: 13.27
Part 6: 8.25
Part 9: 13.92
Part 12: 18.15


In [205]:
stock_part_allocation = {}

for j in stock_dict.keys():
    if stock_vars_results[j].varValue != 0:
        stock_part_allocation[j] = []
    for i in part_dict.keys():
        if part_vars_results[i][j].varValue != 0:
            stock_part_allocation[j].append(i)

stock_part_allocation

{0: [4, 5, 11, 18],
 1: [7, 15, 17],
 2: [1, 3, 13],
 3: [2, 10],
 6: [8, 14, 16],
 7: [0, 6, 9, 12]}

In [210]:
# Double checking results by calculating excess material per piece of stock and overall excess
total_excess = 0
for j, ilist in stock_part_allocation.items():
    stock_excess = stock_dict[j]
    for i in ilist:
        stock_excess = stock_excess - part_dict[i]
    print(stock_excess)
    total_excess = total_excess + stock_excess
print(total_excess)

7.430000000000007
12.680000000000003
4.080000000000005
5.25
7.010000000000002
0.4100000000000037
36.86000000000002
